In [4]:
# pip install rankfm

In [5]:
# pip install implicit

In [8]:
import pandas as pd
# masterdf = pd.read_csv('s3a://hluan/hm/sampled_10_users_transactions.csv', dtype={"article_id": "str"})
masterdf = pd.read_csv('transactions_train.csv', dtype={"article_id": "str"})


In [9]:
from datetime import datetime
masterdf['t_dat'] = pd.to_datetime(masterdf['t_dat'])

In [10]:
masterdf['t_dat'] = datetime(2020, 9, 22) - masterdf['t_dat']
masterdf['t_dat'] = (masterdf['t_dat'].dt.days / 7).astype('int')

In [11]:
masterdf['t_dat'] = 104 - masterdf['t_dat']

In [12]:
master_df_part = masterdf[['t_dat', 'customer_id', 'article_id', 'price']]

In [13]:
master_df_part['rating'] = 1

In [14]:
interactions_dict = master_df_part.groupby(['customer_id', 
                                      'article_id',
                                      't_dat'])[['rating', 'price']].sum().reset_index()


In [15]:
articles = pd.read_csv('articles.csv', dtype={"article_id": "str"})

In [17]:
articles.columns


Index(['article_id', 'product_code', 'prod_name', 'product_type_no',
       'product_type_name', 'product_group_name', 'graphical_appearance_no',
       'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
       'perceived_colour_value_id', 'perceived_colour_value_name',
       'perceived_colour_master_id', 'perceived_colour_master_name',
       'department_no', 'department_name', 'index_code', 'index_name',
       'index_group_no', 'index_group_name', 'section_no', 'section_name',
       'garment_group_no', 'garment_group_name', 'detail_desc'],
      dtype='object')

In [18]:
item_features = pd.get_dummies(articles[['article_id', 'product_type_name', 'product_group_name',
                                        'graphical_appearance_name', 'colour_group_name', 'department_name', 
                                         'index_group_name', 'section_name','garment_group_name']], 
                               columns=['product_type_name', 'product_group_name',
                                        'graphical_appearance_name', 'colour_group_name', 'department_name', 
                                         'index_group_name', 'section_name','garment_group_name'])


In [58]:
interactions = interactions_dict[['customer_id', 
                                      'article_id',
                                      't_dat', 'rating']][interactions_dict['t_dat'].isin([103, 104])]

In [22]:
interactions

,customer_id,article_id,t_dat,rating,random
105,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007,103,1,0.281678
129,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,0448509014,103,1,0.529129
131,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,0719530003,103,1,0.835896
1155,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,0734592001,103,1,0.740475
2225,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,0621381012,103,1,0.514551
...,...,...,...,...,...
28245583,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,0873276003,103,1,0.450946
28245585,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,0881919001,103,2,0.546509
28245597,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,0905365002,103,2,0.950806
28245599,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,0910601003,103,1,0.420355


In [59]:
import numpy as np
import pandas as pd

valid_pct = 0.25
interactions['random'] = np.random.random(size=len(interactions))

# train_mask = interactions['random'] <  (1 - valid_pct)
# valid_mask = interactions['random'] >= (1 - valid_pct)
train_mask = interactions['t_dat'] == 103
valid_mask = interactions['t_dat'] == 104
                          
interactions_train = interactions[train_mask]
interactions_valid = interactions[valid_mask]

sample_weight_train = np.log2(interactions_train['rating'] + 1)
sample_weight_valid = np.log2(interactions_valid['rating'] + 1)

interactions_train = interactions_train[['customer_id', 'article_id']]
interactions_valid = interactions_valid[['customer_id', 'article_id']]

train_users = np.sort(interactions_train.customer_id.unique())
valid_users = np.sort(interactions_valid.customer_id.unique())
cold_start_users = set(valid_users) - set(train_users)

train_items = np.sort(interactions_train.article_id.unique())
valid_items = np.sort(interactions_valid.article_id.unique())
cold_start_items = set(valid_items) - set(train_items)

item_features_train = item_features[item_features.article_id.isin(train_items)]
item_features_valid = item_features[item_features.article_id.isin(valid_items)]

In [60]:


print("train shape: {}".format(interactions_train.shape))
print("valid shape: {}".format(interactions_valid.shape))

print("\ntrain weights shape: {}".format(sample_weight_train.shape))
print("valid weights shape: {}".format(sample_weight_valid.shape))

print("\ntrain users: {}".format(len(train_users)))
print("valid users: {}".format(len(valid_users)))
# print("cold-start users: {}".format(cold_start_users))

print("\ntrain items: {}".format(len(train_items)))
print("valid items: {}".format(len(valid_items)))
print("number of cold-start items: {}".format(len(cold_start_items)))

print("\ntrain item features: {}".format(item_features_train.shape))
print("valid item features: {}".format(item_features_valid.shape))

train shape: (227910, 2)
valid shape: (213728, 2)

train weights shape: (227910,)
valid weights shape: (213728,)

train users: 72019
valid users: 68984

train items: 18611
valid items: 17986
number of cold-start items: 4029

train item features: (18611, 563)
valid item features: (17986, 563)


In [29]:
from rankfm.rankfm import RankFM
from rankfm.evaluation import hit_rate, reciprocal_rank, discounted_cumulative_gain, precision, recall, diversity

In [61]:
model = RankFM(factors=50, loss='warp', max_samples=50, alpha=0.01, learning_rate=0.1, learning_schedule='invscaling')

In [62]:
model.fit(interactions_train, sample_weight=sample_weight_train, epochs=30, verbose=True)


training epoch: 0
log likelihood: -126412.9609375

training epoch: 1
log likelihood: -124370.703125

training epoch: 2
log likelihood: -123891.9296875

training epoch: 3
log likelihood: -122856.3203125

training epoch: 4
log likelihood: -121579.2109375

training epoch: 5
log likelihood: -120173.4375

training epoch: 6
log likelihood: -118248.2578125

training epoch: 7
log likelihood: -116374.0390625

training epoch: 8
log likelihood: -113893.8515625

training epoch: 9
log likelihood: -111574.8671875

training epoch: 10
log likelihood: -108868.5703125

training epoch: 11
log likelihood: -106252.8203125

training epoch: 12
log likelihood: -103699.0078125

training epoch: 13
log likelihood: -101140.8671875

training epoch: 14
log likelihood: -98635.578125

training epoch: 15
log likelihood: -96508.3984375

training epoch: 16
log likelihood: -94409.15625

training epoch: 17
log likelihood: -92512.796875

training epoch: 18
log likelihood: -90829.2734375

training epoch: 19
log likelihood:

In [63]:
%%time
scores = model.predict(interactions_valid, cold_start='nan') 

CPU times: user 344 ms, sys: 0 ns, total: 344 ms
Wall time: 343 ms


In [64]:
%%time
valid_recs = model.recommend(valid_users, n_items=12, filter_previous=False, cold_start='nan')

CPU times: user 5min 16s, sys: 247 ms, total: 5min 16s
Wall time: 5min 16s


In [65]:
print(valid_recs.shape)
valid_recs.head()

(68984, 12)


,0,1,2,3,4,5,6,7,8,9,10,11
00039306476aaf41a07fed942884f16b30abfa83a2a8bea972019098d6406793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf4672f30b3e622fec55,0621381012,0805947001,0761406001,0448509014,0880017001,0536139068,0640021012,0889036004,0572998001,0918292001,0896837001,0547780003
000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed6396773839f6bf71a9,0556255001,0918292001,0732842021,0732842015,0918292011,0908799002,0864216001,0740519002,0816588001,0732842014,0909370001,0874113026
000525e3fe01600d717da8423643a8303390a055c578ed8a97256600baf54565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00077dbd5c4a4991e092e63893ccf29294a9d5c46e85010e95f2fc10bf9437a4,0929744001,0933989002,0865799006,0865799005,0924243001,0456163060,0909370001,0918522001,0573085028,0706016001,0846181003,0909916002


In [ ]:
valid_recs.

In [66]:
k = 12

In [67]:
most_popular = interactions_train.groupby('article_id')['customer_id'].count().sort_values(ascending=False)[:k]
most_popular

article_id
0909370001    1052
0918522001     652
0865799006     651
0924243001     627
0809238001     506
0918292001     498
0751471001     498
0448509014     468
0762846027     460
0809238005     456
0673677002     420
0915529003     419
Name: customer_id, dtype: int64

In [68]:
test_user_items = interactions_valid.groupby('customer_id')['article_id'].apply(set).to_dict()
test_user_items = {key: val for key, val in test_user_items.items() if key in set(train_users)}

base_hrt = np.mean([int(len(set(most_popular.index) & set(val)) > 0)                       for key, val in test_user_items.items()])
base_pre = np.mean([len(set(most_popular.index) & set(val)) / len(set(most_popular.index)) for key, val in test_user_items.items()])
base_rec = np.mean([len(set(most_popular.index) & set(val)) / len(set(val))                for key, val in test_user_items.items()])

In [70]:
print("number of test users: {}".format(len(test_user_items)))
print("baseline hit rate: {:.3f}".format(base_hrt))
print("baseline precision: {:.3f}".format(base_pre))
print("baseline recall: {:.3f}".format(base_rec))

number of test users: 12670
baseline hit rate: 0.061
baseline precision: 0.005
baseline recall: 0.025


In [75]:
%%time
model_hrt = hit_rate(model, interactions_valid, k=k)
model_rnk = reciprocal_rank(model, interactions_valid, k=k)
model_pre = precision(model, interactions_valid, k=k)
model_rec = recall(model, interactions_valid, k=k)

CPU times: user 21min 19s, sys: 864 ms, total: 21min 20s
Wall time: 21min 18s


In [76]:
print("model hit rate: {}".format(round(model_hrt, 3)))
print("model reciprocal rank: {}".format(round(model_rnk, 3)))
print("model precision: {}".format(round(model_pre, 3)))
print("model recall: {}".format(round(model_rec, 3)))

model hit rate: 0.196
model reciprocal rank: 0.089
model precision: 0.02
model recall: 0.115


In [43]:
random_user = np.random.choice(valid_users)
print("random user: {}".format(random_user))

random user: 99ca8a271f41118217f8a3ea014b78c48eae8a49e047bf792d2a53ace8e0b166


In [45]:
random_user_recs = valid_recs.loc[random_user]
random_user_recs = articles[articles.article_id.isin(random_user_recs)].set_index('article_id').loc[random_user_recs]
random_user_recs

,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
article_id,,,,,,,,,,,,,,,,,,,,,
0798972013,798972,Bruce skinny 5 pkt fancy,272,Trousers,Garment Lower body,1010004,Check,17,Yellowish Brown,4,...,Trouser,F,Menswear,3,Menswear,20,Contemporary Smart,1009,Trousers,5-pocket trousers in stretch cotton twill with...
0892558004,892558,sexygan,245,Cardigan,Garment Upper body,1010010,Melange,6,Light Grey,1,...,Jersey fancy,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1005,Jersey Fancy,"Fitted, rib-knit cardigan in a soft, fine visc..."
0875329002,875329,Blenda tee,255,T-shirt,Garment Upper body,1010016,Solid,10,White,3,...,Basic 1,D,Divided,2,Divided,51,Divided Basics,1002,Jersey Basic,Fitted top in cotton jersey. Round neckline wi...
0685814003,685814,RICHIE HOOD,308,Hoodie,Garment Upper body,1010016,Solid,10,White,3,...,Heavy Basic Jersey,F,Menswear,3,Menswear,26,Men Underwear,1002,Jersey Basic,Hoodie in sweatshirt fabric made from a cotton...
0905914002,905914,Cardiff Sweater,252,Sweater,Garment Upper body,1010016,Solid,9,Black,4,...,Tops Knitwear,D,Divided,2,Divided,53,Divided Collection,1003,Knitwear,Cropped top in jersey with a soft brushed fini...
0874196003,874196,PINE crewneck cotton cashmere,252,Sweater,Garment Upper body,1010016,Solid,73,Dark Blue,2,...,Knitwear,F,Menswear,3,Menswear,23,Men Suits & Tailoring,1003,Knitwear,"Round-necked jumper in a soft, fine-knit cotto..."
0894788003,894788,Stretchy 3p Lace V-shape Thong,286,Underwear bottom,Underwear,1010021,Lace,9,Black,4,...,Casual Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Thong briefs in lace with a low, V-shaped wais..."
0893059003,893059,Stacey,245,Cardigan,Garment Upper body,1010010,Melange,13,Beige,1,...,Knitwear,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1003,Knitwear,Long cardigan in a soft knit containing some w...
0636207006,636207,Slim 5pkt Lowprice,272,Trousers,Garment Lower body,1010023,Denim,9,Black,4,...,Denim trousers,F,Menswear,3,Menswear,56,Denim Men,1016,Trousers Denim,5-pocket jeans in washed denim with a regular ...


In [47]:
most_similar_items = model.similar_items('0909370001')
most_similar_items = articles.set_index('article_id').loc[most_similar_items]
most_similar_items

,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
article_id,,,,,,,,,,,,,,,,,,,,,
0923128001,923128,FF Hannah dress,265,Dress,Garment Full body,1010021,Lace,11,Off White,1,...,Campaigns,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1023,Special Offers,"Short, A-line dress in recycled polyester lace..."
0905945002,905945,Tinseltown v-neck dress,254,Top,Garment Upper body,1010010,Melange,13,Beige,2,...,Knitwear,A,Ladieswear,1,Ladieswear,6,Womens Casual,1003,Knitwear,Calf-length dress in a soft knit containing so...
0827487003,827487,DONNA DENIM SKIRT,275,Skirt,Garment Lower body,1010023,Denim,9,Black,4,...,Skirts,D,Divided,2,Divided,53,Divided Collection,1012,Skirts,"Short skirt in a cotton weave with a high, fri..."
0777892001,777892,GRETA SKIRT,275,Skirt,Garment Lower body,1010016,Solid,9,Black,4,...,Skirts,D,Divided,2,Divided,53,Divided Collection,1012,Skirts,Short skirt with a high waist and a detachable...
0906633001,906633,Isach Loafer,86,Ballerinas,Shoes,1010016,Solid,9,Black,4,...,Flats,C,Ladies Accessories,1,Ladieswear,64,Womens Shoes,1020,Shoes,Loafers in imitation leather with a decorative...
0570002002,570002,ROY SLIM RN T-SHIRT,255,T-shirt,Garment Upper body,1010016,Solid,10,White,3,...,Light Basic Jersey,F,Menswear,3,Menswear,26,Men Underwear,1002,Jersey Basic,Round-necked T-shirt in soft jersey.
0570003001,570003,VICTOR SLIM VN T-SHIRT,255,T-shirt,Garment Upper body,1010016,Solid,9,Black,4,...,Light Basic Jersey,F,Menswear,3,Menswear,26,Men Underwear,1002,Jersey Basic,V-neck T-shirt in soft cotton jersey.
0793506028,793506,Chris 3pk Fancy,286,Underwear bottom,Underwear,1010001,All over pattern,32,Orange,5,...,Underwear Jersey,F,Menswear,3,Menswear,26,Men Underwear,1017,"Under-, Nightwear",Trunks in stretch cotton jersey with flatlock ...
0903389002,903389,ED Tree HW pleated Trouser,272,Trousers,Garment Lower body,1010016,Solid,8,Dark Grey,4,...,Woven bottoms,A,Ladieswear,1,Ladieswear,2,H&M+,1009,Trousers,Ankle-length trousers in woven fabric with a h...


## LightFM

In [50]:
# pip install lightfm

In [77]:
from lightfm.data import Dataset
from lightfm import LightFM, evaluation

In [78]:
all_users = interactions.customer_id.unique()
all_items = interactions.article_id.unique()
print(len(all_users), len(all_items))

lfm_dataset = Dataset()
lfm_dataset.fit(users=all_users, items=all_items)

lfm_interactions, lfm_weights = lfm_dataset.build_interactions(zip(interactions_train['customer_id'], interactions_train['article_id'], sample_weight_train))
lfm_interactions, lfm_weights

128333 22640


(<128333x22640 sparse matrix of type '<class 'numpy.int32'>'
 	with 227910 stored elements in COOrdinate format>,
 <128333x22640 sparse matrix of type '<class 'numpy.float32'>'
 	with 227910 stored elements in COOrdinate format>)

In [79]:
lfm_model = LightFM(no_components=50, loss='warp', max_sampled=50)
lfm_model

In [80]:
%%time
lfm_model.fit(lfm_interactions, epochs=20, verbose=True)

Epoch: 100%|██████████| 20/20 [00:16<00:00,  1.24it/s]

CPU times: user 16.2 s, sys: 24 ms, total: 16.2 s
Wall time: 16.2 s


In [81]:
lfm_test_interactions, lfm_test_weights = lfm_dataset.build_interactions(zip(interactions_valid['customer_id'], interactions_valid['article_id']))
lfm_test_interactions

<128333x22640 sparse matrix of type '<class 'numpy.int32'>'
	with 213728 stored elements in COOrdinate format>

In [82]:
%%time
lfm_rnk = evaluation.reciprocal_rank(lfm_model, lfm_test_interactions).mean()
lfm_pre = evaluation.precision_at_k(lfm_model, lfm_test_interactions, k=k).mean()
lfm_rec = evaluation.recall_at_k(lfm_model, lfm_test_interactions, k=k).mean()

CPU times: user 4min 50s, sys: 0 ns, total: 4min 50s
Wall time: 4min 50s


In [83]:
print("reciprocal_rank: {:.3f}".format(lfm_rnk))
print("precision: {:.3f}".format(lfm_pre))
print("recall: {:.3f}".format(lfm_rec))

reciprocal_rank: 0.042
precision: 0.009
recall: 0.044


## Implicit BPR

In [73]:
from implicit.bpr import BayesianPersonalizedRanking as BPR

In [74]:
model_bpr = BPR(factors=50, learning_rate=0.1, iterations=30)

In [85]:
from implicit.datasets import lastfm

In [86]:
a, b, c = lastfm.get_lastfm()

0.00B [00:00, ?B/s]

In [91]:
c[0].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [93]:
from implicit.datasets.movielens import get_movielens
titles, ratings = get_movielens("100k")

0.00B [00:00, ?B/s]

In [120]:
titles.shape

(1683,)

In [99]:
import h5py

In [104]:
with h5py.File('movielens_100k.hdf5', "r") as f:
    m = f.get("movie_user_ratings")
    print(m.get('data'))

<HDF5 dataset "data": shape (100000,), type "<f4">


In [103]:
m

<Closed HDF5 group>

In [ ]:
m = coo_matrix(
        (ratings["rating"].astype(np.float32), (ratings["movieId"], ratings["userId"]))
    ).tocsr()

In [130]:
ratings = pd.read_csv("ml-latest-small/ratings.csv")
movies = pd.read_csv("ml-latest-small/movies.csv")

In [124]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [129]:
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix

In [131]:
m = coo_matrix(
        (ratings["rating"].astype(np.float32), (ratings["movieId"], ratings["userId"]))
    ).tocsr()

In [134]:
m.data

array([4. , 4. , 4.5, ..., 3.5, 3.5, 4. ], dtype=float32)

In [135]:
plays = csr_matrix((m.data, m.indices, m.indptr))

In [136]:
plays

<193610x611 sparse matrix of type '<class 'numpy.float32'>'
	with 100836 stored elements in Compressed Sparse Row format>

In [139]:
masterdf

,t_dat,customer_id,article_id,price,sales_channel_id
0,0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,0,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,0,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,0,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2
...,...,...,...,...,...
31788319,104,fff2282977442e327b45d8c89afde25617d00124d0f999...,0929511001,0.059305,2
31788320,104,fff2282977442e327b45d8c89afde25617d00124d0f999...,0891322004,0.042356,2
31788321,104,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,0918325001,0.043203,1
31788322,104,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,0833459002,0.006763,1


In [140]:
customers = pd.read_csv('customers.csv')

In [142]:
customers['customer_id_int'] = customers['customer_id'].astype('category').cat.codes + 1

In [144]:
articles['article_id_int'] = articles['article_id'].astype('category').cat.codes + 1

In [146]:
interactions = interactions.merge(customers[['customer_id', 'customer_id_int']], on='customer_id')\
    .merge(articles[['article_id', 'article_id_int']], on='article_id')

In [148]:
interactions['rating_log'] = np.log2(interactions['rating'] + 1)

In [159]:


train_mask = interactions['t_dat'] == 103
valid_mask = interactions['t_dat'] == 104
                          
interactions_train = interactions[train_mask]
interactions_valid = interactions[valid_mask]

train_users = np.sort(interactions_train.customer_id.unique())
valid_users = np.sort(interactions_valid.customer_id.unique())
cold_start_users = set(valid_users) - set(train_users)

train_items = np.sort(interactions_train.article_id.unique())
valid_items = np.sort(interactions_valid.article_id.unique())
cold_start_items = set(valid_items) - set(train_items)

In [160]:
m = coo_matrix(
        (interactions_train["rating_log"].astype(np.float32), (interactions_train["article_id_int"], interactions_train["customer_id_int"]))
    ).tocsr()

In [161]:
m_valid = coo_matrix(
        (interactions_valid["rating_log"].astype(np.float32), (interactions_valid["article_id_int"], interactions_valid["customer_id_int"]))
    ).tocsr()

In [162]:
ratings = csr_matrix((m.data, m.indices, m.indptr))

In [163]:
ratings_valid = csr_matrix((m_valid.data, m_valid.indices, m_valid.indptr))

In [152]:
movies = np.array(articles['article_id'])

In [153]:
movies

array(['0108775015', '0108775044', '0108775051', ..., '0956217002',
       '0957375001', '0959461001'], dtype=object)

In [164]:
user_ratings = ratings.T.tocsr()

In [170]:
user_ratings_valid = ratings_valid.T.tocsr()

In [165]:
model_bpr = BPR(factors=50, learning_rate=0.01, iterations=100)

In [166]:
%%time
model_bpr.fit(user_ratings)

  0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 23.3 s, sys: 244 ms, total: 23.5 s
Wall time: 2.44 s


In [169]:
%%time
bpr_rnk = evaluation.reciprocal_rank(model_bpr, m_valid).mean()
bpr_pre = evaluation.precision_at_k(model_bpr, m_valid, k=k).mean()
bpr_rec = evaluation.recall_at_k(model_bpr, m_valid, k=k).mean()

AttributeError: 'BayesianPersonalizedRanking' object has no attribute 'predict_rank'

In [172]:
batch_size = len(valid_users)
to_generate = np.arange(len(valid_users))
for startidx in range(0, len(to_generate), batch_size):
    batch = to_generate[startidx : startidx + batch_size]
    print(batch)
    ids, scores = model_bpr.recommend(
        batch, user_ratings_valid, filter_already_liked_items=False
    )

[    0     1     2 ... 68981 68982 68983]


In [175]:
valid_pred = 

array(['00039306476aaf41a07fed942884f16b30abfa83a2a8bea972019098d6406793',
       '0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf4672f30b3e622fec55',
       '000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed6396773839f6bf71a9',
       ...,
       'fffae8eb3a282d8c43c77dd2ca0621703b71e90904dfde2189bdd644f59071dd',
       'fffd870c6324ad3bda24e4d6aeae221c199479086bfdfd0eb40a3065971d0e01',
       'fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20e02ce5d1e58a8f700b'],
      dtype=object)

In [176]:
ids

array([[    54,  97059, 100727, ..., 104653, 104407,  94096],
       [    54,  97059, 100727, ..., 104653, 104407,  94096],
       [    54,  97059, 100727, ..., 104653, 104407,  94096],
       ...,
       [    54,  97059, 100727, ..., 104653, 104407,  94096],
       [102553,  76473,  78550, ...,  76472, 104149, 104194],
       [101136,     54,  94096, ...,  82781, 100727,  97694]], dtype=int32)

In [183]:
articles[['article_id_int', 'article_id']]

{'article_id_int': {0: 1,
  1: 2,
  2: 3,
  3: 4,
  4: 5,
  5: 6,
  6: 7,
  7: 8,
  8: 9,
  9: 10,
  10: 11,
  11: 12,
  12: 13,
  13: 14,
  14: 15,
  15: 16,
  16: 17,
  17: 18,
  18: 19,
  19: 20,
  20: 21,
  21: 22,
  22: 23,
  23: 24,
  24: 25,
  25: 26,
  26: 27,
  27: 28,
  28: 29,
  29: 30,
  30: 31,
  31: 32,
  32: 33,
  33: 34,
  34: 35,
  35: 36,
  36: 37,
  37: 38,
  38: 39,
  39: 40,
  40: 41,
  41: 42,
  42: 43,
  43: 44,
  44: 45,
  45: 46,
  46: 47,
  47: 48,
  48: 49,
  49: 50,
  50: 51,
  51: 52,
  52: 53,
  53: 54,
  54: 55,
  55: 56,
  56: 57,
  57: 58,
  58: 59,
  59: 60,
  60: 61,
  61: 62,
  62: 63,
  63: 64,
  64: 65,
  65: 66,
  66: 67,
  67: 68,
  68: 69,
  69: 70,
  70: 71,
  71: 72,
  72: 73,
  73: 74,
  74: 75,
  75: 76,
  76: 77,
  77: 78,
  78: 79,
  79: 80,
  80: 81,
  81: 82,
  82: 83,
  83: 84,
  84: 85,
  85: 86,
  86: 87,
  87: 88,
  88: 89,
  89: 90,
  90: 91,
  91: 92,
  92: 93,
  93: 94,
  94: 95,
  95: 96,
  96: 97,
  97: 98,
  98: 99,
  99: 100,


In [184]:
articles.iloc[54]['article_id']

'0148033006'

In [187]:
valid_pred = pd.DataFrame()
valid_pred['custome_id'] = valid_users
valid_pred['preds'] = [" ".join([str(i) for i in id]) for id in ids]
valid_pred['scores'] = [" ".join([str(i) for i in id]) for id in scores]

In [188]:
valid_pred['preds'] = valid_pred['preds'].str.split()
valid_pred['scores'] = valid_pred['scores'].str.split()

In [192]:
def pred_replace(preds):
    new_preds = []
    for p in preds:
        new_preds.append(articles.iloc[int(p)]['article_id'])
    return new_preds

In [193]:
valid_pred['preds_articles'] = valid_pred['preds'].apply(lambda x: pred_replace(x))

In [196]:
interactions_valid.groupby('customer_id')['article_id'].apply(list).reset_index()

,customer_id,article_id
0,00039306476aaf41a07fed942884f16b30abfa83a2a8be...,[0624486001]
1,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,[0827487003]
2,000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed...,"[0640021019, 0757926001, 0788575004]"
3,000525e3fe01600d717da8423643a8303390a055c578ed...,[0874110016]
4,00077dbd5c4a4991e092e63893ccf29294a9d5c46e8501...,"[0448509014, 0158340001, 0486639003, 079936502..."
...,...,...
68979,fffa67737587e52ff1afa9c7c6490b5eb7acbc439fe82b...,"[0911870004, 0874816003]"
68980,fffa7d7799eb390a76308454cbdd76e473d65b1497fbe4...,"[0861803014, 0849886010]"
68981,fffae8eb3a282d8c43c77dd2ca0621703b71e90904dfde...,"[0894320003, 0865624003, 0918200001, 079789200..."
68982,fffd870c6324ad3bda24e4d6aeae221c199479086bfdfd...,"[0761269001, 0750423010]"


In [199]:
valid_pred.columns = ['customer_id', 'preds', 'scores', 'preds_articles']

In [200]:
valid_pred = valid_pred.merge(interactions_valid.groupby('customer_id')['article_id'].apply(list).reset_index(), on='customer_id')

In [201]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [203]:
mapk(valid_pred['article_id'], valid_pred['preds_articles'], k=12)

5.830231389997146e-05